In [14]:
import pandas as pd
import itertools

# File path to load

In [15]:
file_path = 'transactions_with_parents.csv'

 # Parameters

In [31]:
IS_PRINT_ = False
MIN_SUP = 140
MIN_CONF = 0.3

# Apriori algorithm


In [32]:
class AprioriAlgorithm:
    def __init__(self, min_support, min_confidence):
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.df_transaction_dataset = None
        self.item_counts = None
        self.frequent_itemsets = {}
        self.transactions_taxonomy_elements_frequency = None

    def load_data(self, file_path,size=2000):
        self.df_transaction_dataset = pd.read_csv(file_path)
        self.df_transaction_dataset = self.df_transaction_dataset.head(size)

    ########################################
    # Frequency Counting
    ########################################
    def frequency_counting(self):
        item_counts = self.df_transaction_dataset[[col for col in self.df_transaction_dataset.columns if col.isdigit() and len(col) == 4]].sum()
        unique_combinations = set()
        combination_counts = {}

        for index, row in self.df_transaction_dataset.iterrows():
            for col in self.df_transaction_dataset.columns:
                if col.isdigit() and len(col) == 3:
                    value = row[col]
                    if value != 0:
                        combination_key = (col, value)
                        if combination_key in unique_combinations:
                            combination_counts[combination_key] += 1
                        else:
                            unique_combinations.add(combination_key)
                            combination_counts[combination_key] = 1

        combination_counts_series = pd.Series(combination_counts)
        transactions_taxonomy_elements_frequency = pd.concat([item_counts, combination_counts_series])

        return transactions_taxonomy_elements_frequency


    ########################################
    # Generate Frequent k-itemsets
    ########################################
    def generate_candidates(self, prev_itemsets, k):
        candidates = set()
        for itemset1 in prev_itemsets:
            for itemset2 in prev_itemsets:
                if len(itemset1.union(itemset2)) == k:
                    candidates.add(itemset1.union(itemset2))
        return candidates

    def prune_itemsets(self, itemsets, rejected_itemsets):
        pruned_itemsets = {}
        if rejected_itemsets:
            itemsets = set(itemset for itemset in itemsets if itemset not in rejected_itemsets)
        for itemset in itemsets:
            support = sum(
                all(
                    (item[0] in row and row[item[0]] == item[1])
                    if isinstance(item, tuple)
                    else (item in row and row[item] != 0)
                    for item in itemset
                )
                for _, row in self.df_transaction_dataset.iterrows()
            )
            if support >= self.min_support:
                pruned_itemsets[itemset] = support
            else:
                rejected_itemsets.add(itemset)
        return pruned_itemsets

    def generate_frequent_itemsets(self):
        frequent_1_itemsets = {}
        rejected_itemsets = set()

        for item, count in self.transactions_taxonomy_elements_frequency.items():
            if count >= self.min_support:
                frequent_1_itemsets[frozenset([item])] = count

        self.frequent_itemsets[1] = frequent_1_itemsets

        k = 2
        while self.frequent_itemsets[k - 1]:
            candidate_itemsets = self.generate_candidates(self.frequent_itemsets[k - 1].keys(), k)
            pruned_itemsets = self.prune_itemsets(candidate_itemsets, rejected_itemsets)
            self.frequent_itemsets[k] = pruned_itemsets
            k += 1

    ########################################
    # Generate Association Rules
    ########################################
    def generate_association_rules(self):
        association_rules = []
        frequent_itemsets_to_create_rules = self.frequent_itemsets[len(self.frequent_itemsets) - 2]

        for itemset in frequent_itemsets_to_create_rules.items():
            support_without_tuple = itemset[1]
            itemset = itemset[0]
            new_itemset = True
            last_support_for_tuple = None
            subsets = self.get_subsets(itemset)

            for subset in subsets:
                antecedent = subset
                consequent = itemset - subset
                if len(antecedent) >= 2 or isinstance(antecedent, tuple) or any(
                        isinstance(x, tuple) for x in consequent):
                    antecedent_support = sum(
                        all(
                            (row[item[0]] > item[1])
                            if isinstance(item, tuple)
                            else (row[item] == 1)
                            for item in antecedent
                        )
                        for _, row in self.df_transaction_dataset.iterrows()
                    )

                    if new_itemset == True:
                        new_itemset = False
                        base_support = sum(
                            all(
                                (row[item[0]] > item[1])
                                if isinstance(item, tuple)
                                else (row[item] == 1)
                                for item in itemset
                            )
                            for _, row in self.df_transaction_dataset.iterrows()
                        )

                        last_support_for_tuple = base_support
                    support = last_support_for_tuple
                    if antecedent_support > 0:
                        confidence = support / antecedent_support
                        if confidence >= self.min_confidence:
                            association_rules.append((antecedent, consequent, support, confidence))
                else:
                    antecedent_support = self.frequent_itemsets[len(antecedent)][antecedent]
                    confidence = support_without_tuple / antecedent_support
                    if confidence >= self.min_confidence:
                        association_rules.append((antecedent, consequent, support_without_tuple, confidence))

        return association_rules

    def get_subsets(self, itemset):
        subsets = []
        itemset = list(itemset)
        num_items = len(itemset)

        for i in range(1, num_items):
            subsets.extend(itertools.combinations(itemset, i))

        return [frozenset(subset) for subset in subsets]

    def run_algorithm(self, file_path,dataset_size=2000):
        self.load_data(file_path,dataset_size)
        self.transactions_taxonomy_elements_frequency = self.frequency_counting()
        self.generate_frequent_itemsets()
        association_rules = self.generate_association_rules()
        return self.transactions_taxonomy_elements_frequency, association_rules

In [33]:
class AprioriAlgorithmRunner:
    def __init__(self, min_support, min_confidence):
        self.apriori_algorithm = AprioriAlgorithm(min_support, min_confidence)

    def run_algorithm(self, file_path, dataset_size=2000):
        return self.apriori_algorithm.run_algorithm(file_path, dataset_size)

# Run algorithm

In [34]:
runner = AprioriAlgorithmRunner(min_support=MIN_SUP, min_confidence=MIN_CONF)
transactions_taxonomy_elements_frequency, association_rules = runner.run_algorithm(file_path=file_path,dataset_size = 5000)

In [35]:
print("Frequency Counting Results:")
print(transactions_taxonomy_elements_frequency)

Frequency Counting Results:
1001          19
1002         175
1003           2
1004           8
1005          47
            ... 
(100, 14)      1
(132, 3)       1
(213, 1)       4
(130, 6)       1
(303, 3)       1
Length: 1394, dtype: int64


In [36]:
print("\nAssociation Rules:")
for antecedent, consequent, support, confidence in association_rules:
    antecedent_str = ', '.join([str(item) for item in antecedent])
    consequent_str = ', '.join([str(item) for item in consequent])
    print(f"Antecedent: {antecedent_str}, Consequent: {consequent_str}, Support: {support}, Confidence: {confidence}")


Association Rules:
Antecedent: ('132', 1), Consequent: ('130', 1), ('100', 2), ('150', 1), Support: 28, Confidence: 0.5384615384615384
Antecedent: ('132', 1), ('150', 1), Consequent: ('130', 1), ('100', 2), Support: 28, Confidence: 1.0
Antecedent: ('130', 1), ('132', 1), Consequent: ('100', 2), ('150', 1), Support: 28, Confidence: 0.5384615384615384
Antecedent: ('100', 2), ('132', 1), Consequent: ('130', 1), ('150', 1), Support: 28, Confidence: 0.6363636363636364
Antecedent: ('130', 1), ('132', 1), ('150', 1), Consequent: ('100', 2), Support: 28, Confidence: 1.0
Antecedent: ('100', 2), ('132', 1), ('150', 1), Consequent: ('130', 1), Support: 28, Confidence: 1.0
Antecedent: ('130', 1), ('100', 2), ('132', 1), Consequent: ('150', 1), Support: 28, Confidence: 0.6363636363636364
Antecedent: ('154', 1), Consequent: 1113, ('100', 1), ('150', 1), Support: 17, Confidence: 0.7727272727272727
Antecedent: 1113, ('154', 1), Consequent: ('100', 1), ('150', 1), Support: 17, Confidence: 1.0
Antecede